In [81]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from skl2onnx.convert import convert_sklearn
from skl2onnx.common.data_types import StringTensorType
from onnxruntime import InferenceSession
from onnxmltools.utils import save_model

In [3]:
cats = ['rec.motorcycles', 'sci.electronics']
training_data = fetch_20newsgroups(subset='train', categories=cats)
test_data = fetch_20newsgroups(subset='test', categories=cats)

In [34]:
X_train, y_train = np.array(training_data.data), training_data.target
X_test, y_test = np.array(test_data.data), test_data.target

In [74]:
model = Pipeline([('tfidf', CountVectorizer()), ('predictor', MLPClassifier())])
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidf', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False))])

In [75]:
np.mean(model.predict(X_test) == y_test)

0.9823008849557522

In [76]:
model_onnx = convert_sklearn(model, 'newsgroup', [('input', StringTensorType([None]))])
save_model(model_onnx, 'news.onnx')
sess = InferenceSession('news.onnx')

WARNING - The maximum opset needed by this model is only 10.
WARNING - The maximum opset needed by this model is only 1.
WARNING - The maximum opset needed by this model is only 1.


In [77]:
res = sess.run(None, input_feed={'input': X_test})

In [78]:
np.mean(res[0] == model.predict(X_test))

1.0

In [80]:
np.mean(np.isclose(list(map(lambda x: [x[0], x[1]], res[1])), model.predict_proba(X_test), atol=1e-7))

1.0